# KrissyEVDisparity Analysis
----


In [289]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Census Data and DOE Data results into DataFrame

In [290]:
EVStation = pd.read_csv("EVstation_data.csv")
EVStation['Zipcode']=EVStation['Zipcode'].astype(int)
state_charger_df= EVStation.groupby('State').size().reset_index(name='counts')
state_charger_df


State  counts
0     AZ     803
1     CA   13899
2     CO    1489
3     ID     136
4     MT      70
5     NM     171
6     NV     449
7     OR     939
8     UT     845
9     WA    1686
10    WY      64

In [291]:
state_census= pd.read_csv("state_census_df.csv")
state_census = state_census.dropna()

state_census['Poverty Rate']=state_census['Poverty Rate'].astype(int)
# state_census = state_census['Average Vehicles per Household'].dropna()
# state_census['Average Vehicles per Household']=state_census['Average Vehicles per Household'].astype(int)
state_census.head()
# len(state_census)

State ID  Zipcode  Population  Number of Households  Household Income  \
0        30    59001      1558.0                 734.0           58482.0   
1        30    59002       127.0                  47.0           37917.0   
2        30    59003      1359.0                 433.0           46838.0   
3        30    59006      1461.0                 518.0           44750.0   
4        30    59007       172.0                  66.0          105625.0   

   Poverty Count  Poverty Rate  Total Commute Time  Average Commute Time  \
0           14.0             0               679.0              0.925068   
1           33.0            25                52.0              1.106383   
2          362.0            26               490.0              1.131640   
3          377.0            25               543.0              1.048263   
4           13.0             7                72.0              1.090909   

   Total Vehicles  Average Vehicles per Household State  
0           545.0                        0.742507    MT  
1            45.0                        0.957447    MT  
2           325.0                        0.750577    MT  
3           485.0                        0.936293    MT  
4            65.0                        0.984848    MT

In [292]:
zipmerge=pd.merge(state_census,EVStation, how = "inner", on="Zipcode")
zipmerge.head()

State ID  Zipcode  Population  Number of Households  Household Income  \
0        30    59011      3120.0                1347.0           49754.0   
1        30    59024       444.0                 200.0           53136.0   
2        30    59030      1211.0                 684.0           51694.0   
3        30    59047     12728.0                5904.0           53777.0   
4        30    59068      3355.0                1654.0           48821.0   

   Poverty Count  Poverty Rate  Total Commute Time  Average Commute Time  \
0          179.0             5              1195.0              0.887157   
1           35.0             7               116.0              0.580000   
2          140.0            11               716.0              1.046784   
3         1569.0            12              5733.0              0.971037   
4          345.0            10              1551.0              0.937727   

   Total Vehicles  Average Vehicles per Household State_x State_y        city  \
0           950.0                        0.705271      MT      MT  Big Timber   
1            85.0                        0.425000      MT      MT      Custer   
2           545.0                        0.796784      MT      MT    Gardiner   
3          4685.0                        0.793530      MT      MT  Livingston   
4          1150.0                        0.695284      MT      MT   Red Lodge   

    latitude   longitude  
0  45.836300 -109.943400  
1  46.129478 -107.553221  
2  45.029727 -110.707820  
3  45.659342 -110.558047  
4  45.190425 -109.247648

In [293]:
statemerge=pd.merge(state_census,EVStation, how = "inner", on="State")
statemerge.head()

State ID  Zipcode_x  Population  Number of Households  Household Income  \
0        30      59001      1558.0                 734.0           58482.0   
1        30      59001      1558.0                 734.0           58482.0   
2        30      59001      1558.0                 734.0           58482.0   
3        30      59001      1558.0                 734.0           58482.0   
4        30      59001      1558.0                 734.0           58482.0   

   Poverty Count  Poverty Rate  Total Commute Time  Average Commute Time  \
0           14.0             0               679.0              0.925068   
1           14.0             0               679.0              0.925068   
2           14.0             0               679.0              0.925068   
3           14.0             0               679.0              0.925068   
4           14.0             0               679.0              0.925068   

   Total Vehicles  Average Vehicles per Household State       city  Zipcode_y  \
0           545.0                        0.742507    MT   Missoula      59801   
1           545.0                        0.742507    MT    Bozeman      59771   
2           545.0                        0.742507    MT   Missoula      59401   
3           545.0                        0.742507    MT   Billings      59102   
4           545.0                        0.742507    MT  Kalispell      59901   

    latitude   longitude  
0  46.845621 -114.023339  
1  45.677719 -111.036339  
2  46.872089 -113.991729  
3  45.754983 -108.568626  
4  48.163400 -114.286926

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [294]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [295]:
# Store 'Lat' and 'Lng' into  locations 
EVStation = EVStation.dropna()
locations = EVStation[["latitude", "longitude"]].astype(float)


In [296]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, #weights= humidity, 
                                 
                                 dissipating=False, max_intensity=3000,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [297]:
# Create bank symbol layer
# locationslist = EVStation[["latitude", "longitude"]]
bank_rate = census_data_complete["counts"].tolist()
station_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2
    info_box_content=[f"Bank amount: {latitude,longitude}" for location in locations]
)


fig = gmaps.figure()
fig.add_layer(station_layer)

fig

SyntaxError: invalid syntax (<ipython-input-297-6d3f5931eafd>, line 7)

In [ ]:


# hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# # Display figure

# fig.add_layer(hotel_layer)

# fig


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#use .loc

#.head()
#caveat: make sure your ideal isn't too ideal, meaning check the length of resulting dataframe so it has rows
#call this dataframe hotel_df

hotel_df=state_census.loc[(state_census["Poverty Rate"] > 40)&(state_census["Household Income"] > 1)]
hotel_df.head()

In [ ]:
# set up additional columns to hold information
# hotel_df['Hotel Name'] = ""

# hotel_df.head()

In [ ]:
#how do we get closest station here? does gmaps have EV charging station?

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "radius": 5000,
#     "type": "lodging",
#     "key": g_key,
# }
# # use iterrows to iterate through pandas dataframe
# for index, row in hotel_df.iterrows():
    
#     lat =row["latitude"]
#     lon =row["longitude"]

#     # add keyword to params dict
#     params['location'] = f'{lat},{lon}'

#     # assemble url and make API request
#     print(f"Retrieving Results for Index {index}: {row['City']}.")
#     response = requests.get(base_url, params=params).json()
    
#     # extract results
#     results = response['results']
    
#     try:
#         print(f"Closest {row['City']} is {results[0]['name']}.")
        
#         hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

In [ ]:
hotel_df


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
#Convert Hotel df to list
# hotel_rate = hotel_df[["Lat","Lng"]]

In [ ]:
# Add marker layer ontop of heat map
# Create bank symbol layer

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure

fig.add_layer(hotel_layer)

fig



In [ ]:
Number of stations per household
#Create gender data frame
# # state_df = pd.DataFrame(EVStation['State'].value_counts())
# # gender_data_df.head()
# # Chargingstation=state_census[['State','Household Income','Number of Households']]

# # state_census.groupby(['State'])
# max_age = purchase_data_df['Age'].max()
# print(max_age)
# min_age = purchase_data_df['Age'].min()
# print(min_age)

# bins = [0, 10, 14, 19, 24, 29, 34, 39, 44, 49]
# group_names = ['< 10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49']

# SN_gen_age_df['Age Group'] = pd.cut(SN_gen_age_df['Age'], bins, labels=group_names)
# SN_gen_age_df

# countcharging= EVStation.groupby(['State'])
# # countcharging.head()
# numcharg = EVStation['latitude'].count()
# numcharg
# # numstationsum=pd.DataFrame({'state':countcharging,'count charging':numcharg})
# # numstationsum

# # numberofstations = EVStation.value_counts()
# # numberofstations 
# # # #Summarize in One Chart
# # #percentage_players = ['bananas', 'peaches','pin']
# # gender_data_df['Percentage of Players']= percentage_players
# # gender_data_df['Percentage of Players']=gender_data_df['Percentage of Players'].map("{:,.2%}".format)
# # gender_data_df.head()
